# Optimizer Monitor

In [ ]:
import matplotlib.pyplot as plt
from molass_data import SAMPLE4
from molass_legacy._MOLASS.SerialSettings import clear_temporary_settings, set_setting
from molass_legacy.Batch.LiteBatch import LiteBatch
from molass_legacy.Models.ModelUtils import plot_component_curves
clear_temporary_settings()
batch = LiteBatch()
sd = batch.load_data(SAMPLE4)
batch.prepare(sd)
uv_x, uv_y, xr_x, xr_y, baselines = batch.get_curve_xy(return_baselines=True, debug=False)
uv_y_ = uv_y - baselines[0]
xr_y_ = xr_y - baselines[1]
uv_peaks, xr_peaks = batch.get_modeled_peaks(uv_x, uv_y_, xr_x, xr_y_, affine=True, min_area_prop=0.05)
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))
fig.suptitle("Impl Plot for %s" % SAMPLE4)
plot_component_curves(ax1, uv_x, uv_y, uv_peaks, baselines[0], color='blue', affine=True)
plot_component_curves(ax2, xr_x, xr_y, xr_peaks, baselines[1], color='orange', affine=True)
fig.tight_layout()
plt.show()

In [ ]:
import os
def prepare_for_v2_optimization(data_folder, analysis_folder=None):
    from molass_legacy._MOLASS.SerialSettings import clear_temporary_settings
    from molass_legacy.KekLib.BasicUtils import Struct
    from molass_legacy.Batch.StandardProcedure import StandardProcedure
    from molass_legacy.Trimming.PreliminaryRecognition import PreliminaryRecognition
    from molass_legacy.SecSaxs.DataTreatment import DataTreatment
    from molass_legacy.RgProcess.RgCurve import RgCurve
    from molass_legacy.Optimizer.OptDataSets import OptDataSets

    clear_temporary_settings()
    if analysis_folder is None:
        analysis_folder = './temp_analysis_folder'
    os.makedirs(analysis_folder, exist_ok=True)
    set_setting('analysis_folder', analysis_folder)
    optimized_folder = os.path.join(analysis_folder, 'optimized')
    os.makedirs(optimized_folder, exist_ok=True)
    rg_curve_folder = os.path.join(analysis_folder, 'rg-curve')
    os.makedirs(rg_curve_folder, exist_ok=True)

    sp = StandardProcedure()
    sd = sp.load_old_way(data_folder)

    pre_recog = PreliminaryRecognition(sd)
    treat = DataTreatment(route="v2", trimming=1, correction=1)
    treat_path = os.path.join(optimized_folder, 'treatment.txt')
    treat.save(treat_path)
 
    sd_copy = treat.get_treated_sd(sd, pre_recog)
    D, E, qv, ecurve = sd_copy.get_xr_data_separate_ly()
    rg_curve = RgCurve(qv, ecurve, D, E)
    rg_curve.export(rg_curve_folder)

    dsets = OptDataSets(sd, sd_copy, rg_folder=rg_curve_folder)

    return Struct(sd=sd_copy, optimized_folder=optimized_folder, rg_curve=rg_curve, dsets=dsets)

result = prepare_for_v2_optimization(SAMPLE4)
result